# 1 Import all packages

In [ ]:
import pandas as pd
import requests
import re
from pytz import timezone #to select current timezone
from datetime import datetime, timedelta # creating timestamps 
from bs4 import BeautifulSoup
from lat_lon_parser import parse    # for decimal coordinates
import sqlalchemy
import pymysql

In [2]:
# sql root and api keys. Utilizes .env file and config.py to configure password management
from config import SQL_ROOT, OPENWEATHER_API_KEY, AIRPORT_GEO_API_KEY, RAPID_API_WBS
from con_cloud import con as cloud_con
from con_local import con as local_con

## 2.1 Dataframe creation for city data

In [3]:
def cities_dataframe(cities):

  city_data = []
  population_data = []

  for city in cities:
    url = f"https://www.wikipedia.org/wiki/{city}"
    response = requests.get(url)
    city_soup = BeautifulSoup(response.content, 'html.parser')

    # extract the relevant information
    city_latitude = city_soup.find(class_="latitude").get_text()
    city_longitude = city_soup.find(class_="longitude").get_text()
    country = city_soup.find(class_="infobox-data").get_text()

    # keep track of data per city
    city_data.append({"city": city,
                    "country": country,
                    "latitude": parse(city_latitude), # latitude in decimal format
                    "longitude": parse(city_longitude), # longitude in decimal format
                    })
    
    # extract the relevant information
    city_population = city_soup.find(string="Population").find_next("td").get_text()
    try:
      city_population_clean = int(city_population.replace(",", ""))
    except:
      city_population_clean = -999
    today = datetime.today().strftime("%Y-%m-%d")
    pop_census = city_soup.find(string = 'Population').find_next('div').get_text()
    try:
      yr_census = re.findall(r'[0-9]{4}', pop_census)[0]
    except:
      yr_census = -999

    # for each city we append a dictionary of values to the list
    population_data.append({"city": city,
                          "population": city_population_clean,
                          "population_timestamp": int(yr_census),
                          "retrieval_timestamp": today
                          })


  return pd.DataFrame(population_data), pd.DataFrame(city_data)

## 2.2 Dataframe creation for weather data

In [ ]:
def get_weather_data(input_city_df):

    
    # Setting up lists for later dataframe creation
    city_id = []
    forecast_time = []
    outlook = []
    temperature = []
    temperature_felt = []
    wind_speed = []
    rain_expected_mm = []
    retrieval_time = []

    # Prepping time and source related references
    berlin_timezone = timezone('Europe/Berlin')
    
    url = "https://api.openweathermap.org/data/2.5/forecast"

    for id in input_city_df['city_id']:
        querystring = {"lat": input_city_df.loc[input_city_df['city_id'] == id, :]['latitude'], "lon": input_city_df.loc[input_city_df['city_id'] == id, :]['longitude'], "appid": OPENWEATHER_API_KEY, "units": "metric"}
        retrieval_timestamp = datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")
        # Reference the sections in the request.
        weather = requests.request("GET", url, params=querystring)
        weather_json = weather.json()

        for element in weather_json['list']:
            city_id.append(id)
            forecast_time.append(element['dt_txt'])
            outlook.append(element['weather'][0]['description'])
            temperature.append(element['main']['temp'])
            temperature_felt.append(element['main']['feels_like'])
            wind_speed.append(element['wind']['speed'])
            try:
                rain_expected_mm.append(element['rain']['3h'])
            except:
                rain_expected_mm.append(0)
            retrieval_time.append(retrieval_timestamp)

    weather_df_dictionary = pd.DataFrame({"city_id": city_id,
                                        "forecast_time": forecast_time,
                                        "outlook": outlook,
                                        "temperature": temperature,
                                        "temperature_felt": temperature_felt,
                                        "wind_speed": wind_speed,
                                        "rain_expected_mm": rain_expected_mm,
                                        "retrieval_time": retrieval_time}
                                    )
    weather_df_dictionary["forecast_time"] = pd.to_datetime(weather_df_dictionary["forecast_time"])
    weather_df_dictionary["retrieval_time"] = pd.to_datetime(weather_df_dictionary["retrieval_time"]) 
    return weather_df_dictionary


## 2.3 Dataframe Creation for Airport Info

In [ ]:
def get_airport_info(input_city_df):
    url = "https://aerodatabox.p.rapidapi.com/airports/search/location"
    cities_airports = []

    for id in input_city_df['city_id']:
        querystring = {"lat":input_city_df.loc[input_city_df['city_id'] == id, :]['latitude'],
                "lon":input_city_df.loc[input_city_df['city_id'] == id, :]['longitude'],
                "radiusKm":"30",
                "limit":"8",
                "withFlightInfoOnly":"True"
                }

        headers = {
        "x-rapidapi-key": RAPID_API_WBS,
        "x-rapidapi-host": "aerodatabox.p.rapidapi.com"
        }

        airport_geo_json = requests.get(url, headers=headers, params=querystring).json()

        city_airports = pd.json_normalize(airport_geo_json['items'])
        city_airports['city_id'] = id
        cities_airports.append(city_airports)

    cities_airports_df = pd.concat(cities_airports, ignore_index=True)
    cities_airports_df.rename(columns = {'name': 'airport_name', 'location.lat': 'latitude', 'location.lon': 'longitude'}, inplace = True)
    return cities_airports_df


In [6]:
# Creating the airports_df for unique icao/iata combination. icao is FK to icao in cities_airports
def create_airport_df(cities_airports_df):
    airports_icao = []
    airports_iata = []
    airports_name = []
    for icao_id in cities_airports_df['icao']:
        if icao_id in cities_airports_df['icao'].unique() and icao_id not in airports_icao:
            airports_icao.append(icao_id)
            airports_iata.append(cities_airports_df.loc[cities_airports_df['icao'] == icao_id]['iata'].reset_index(drop=True)[0])
            airports_name.append(cities_airports_df.loc[cities_airports_df['icao'] == icao_id]['airport_name'].reset_index(drop=True)[0])
    airports_df_dictionary = pd.DataFrame({"icao": airports_icao,
                                        "iata": airports_iata,
                                        "airport_name": airports_name
                                        })
    return airports_df_dictionary

## 2.4 Function for flights dataframe creation

In [ ]:
def get_flights(input_icao_list):
    querystring = {"withLeg":"True",
                "direction":"Arrival",
                "withCancelled":"False",
                "withCodeshared":"True",
                "withCargo":"False",
                "withPrivate":"False",
                "withLocation":"False"
                }

    headers = 	{
                "x-rapidapi-key": RAPID_API_WBS,
                "x-rapidapi-host": "aerodatabox.p.rapidapi.com"
                }

    berlin_timezone = timezone('Europe/Berlin')
    today = datetime.now(berlin_timezone).date()
    tomorrow = (today + timedelta(days=1))
    times = [["00:00","11:59"],
            ["12:00","23:59"]]

    flight_num = []
    departure_icao = []
    arrival_icao = []
    arrival_time = []
    retrieval_time = []


    retrieval_timestamp = datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")

    for airport in input_icao_list:
        for time in times:
            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{airport}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"

            flights_response = requests.get(url, headers=headers, params=querystring)
            try:
                flights_json = flights_response.json()
            except requests.JSONDecodeError:
                continue
            for flight in flights_json['arrivals']:
                flight_num.append(flight['number'])
                try:
                    departure_icao.append(flight['departure']['airport']['icao'])
                except:
                    departure_icao.append('')
                arrival_icao.append(airport)
                arrival_time.append(flight['arrival']['scheduledTime']['local'])
                retrieval_time.append(retrieval_timestamp)
    airports_df_dictionary = pd.DataFrame({"flight_num": flight_num,
                                        "departure_icao": departure_icao,
                                        "arrival_icao": arrival_icao,
                                        "arrival_time": arrival_time,
                                        "retrieval_time": retrieval_time
                                            })
    airports_df_dictionary["arrival_time"] = airports_df_dictionary["arrival_time"].str[:-6]
    airports_df_dictionary["arrival_time"] = pd.to_datetime(airports_df_dictionary["arrival_time"])
    airports_df_dictionary["retrieval_time"] = pd.to_datetime(airports_df_dictionary["retrieval_time"])
    return airports_df_dictionary
        

## 2.5 Function that fetches city and population airports and cities_airports data from a database

In [3]:
def fetch_static_data(connection):
    city_df             = pd.read_sql("city", 
                                     con=connection)
    population_df       = pd.read_sql("population",
                                     con = connection)
    airports_df         = pd.read_sql("airports",    
                                     con = connection)
    cities_airports_df  = pd.read_sql("cities_airports",
                                     con = connection)
    return city_df, population_df, airports_df, cities_airports_df

## 2.7 Function that fetches flights and weather data from a database

In [4]:
def fetch_dynamic_data(connection):
    weather_df  = pd.read_sql("weather", 
                             con=connection)
    flights_df  = pd.read_sql("flights",
                             con = connection)
    return weather_df, flights_df

## 2.8 Function that submits static databases to SQL

In [5]:
def submit_static_data(city_df_in, population_df_in, airports_df_in, cities_airports_df_in, connection):
    city_df_in.to_sql('city',
                  if_exists='append',
                  con=connection,
                  index=False)
    population_df_in.to_sql('population',
                         if_exists='append',
                         con=connection,
                         index=False)
    airports_df_in.to_sql('airports',  
                       if_exists='append',
                       con=connection,
                       index=False)
    cities_airports_df_in.to_sql('cities_airports',
                              if_exists='append',
                              con=connection,
                              index=False)
    return "success"
    

## 2.9 Function that submits weather data to SQL

In [6]:
def submit_weather_data(weather_df_in, connection):
    weather_df_in.to_sql('weather',
                  if_exists='append',
                  con=connection,
                  index=False)
    return "success"

## 2.10 Function that submits flighhts data to SQL

In [7]:
def submit_flights_data(flights_df_in, connection):
    flights_df_in.to_sql('flights',
                  if_exists='append',
                  con=connection,
                  index=False)
    return "success"

## 2.11 Function that automatically parses and updates weather and flights tables

In [17]:
def update_weather_flights(connection):
    city_df, population_df, airports_df, cities_airports_df = fetch_static_data(connection)
    weather_df = get_weather_data(city_df)
    flights_df = get_flights(airports_df['icao'])
    submit_weather_data(weather_df, connection)
    submit_flights_data(flights_df, connection)
    return "success"
    


## 3.1 Filling the dataframe for selected cities

In [ ]:
cities = ["Berlin", "Hamburg", "Munich", "Cologne", "London"]

pop_df, city_df = cities_dataframe(cities)

city_df

In [ ]:
pop_df

Export the city adata to SQL

In [ ]:
city_df.to_sql('city',
                  if_exists='append',
                  con=local_con,
                  index=False)

5

Reimport city table with primary key assignments. Prepare population data for exporting 

In [ ]:
city_from_sql = pd.read_sql("city", con=local_con) # Reimporting

In [ ]:

# Left Join city into population data
population_df = pop_df.merge(city_from_sql,
                                   on = "city",
                                   how = "left")

# Remove duplicate table info
population_df.drop(columns=["city", 'country', 'latitude', 'longitude'], inplace = True)

# Reorder columns to match SQL table generation: city_id, population, population_timestamp, retrieval_timestamp from l to r
population_df = population_df.iloc[:, [-1, 0, 1, 2]]

# Export to SQL population table
population_df.to_sql('population',
                  if_exists='append',
                  con=local_con,
                  index=False)

## 3.2 Filling the weather df (requires city_df to be setup )

In [ ]:
weather_df = get_weather_data(city_from_sql)
weather_df

Exporting the weather dataframe to SQL

In [ ]:
weather_df.to_sql('weather',
                  if_exists='append',
                  con=local_con,
                  index=False)

# 3.3 Filling the cities_airports and airports dataframes. (Requires city_df)

In [ ]:
cities_airports_df = get_airport_info(city_from_sql)
cities_airports_df

In [ ]:
airports_df = create_airport_df(cities_airports_df)
airports_df

In [ ]:
#Export airports_df to SQL
airports_df.to_sql('airports',
                  if_exists='append',
                  con=local_con,
                  index=False)

In [ ]:
#Export cities_airports_df to SQL 
cities_airports_df.to_sql('cities_airports',
                  if_exists='append',
                  con=local_con,
                  index=False)

## 3.4 Filling the database with flight info

In [ ]:
flights_df = get_flights(airports_df['icao'])
flights_df

In [ ]:
#Export flights dataframes to SQL
flights_df.to_sql('flights',
                  if_exists='append',
                  con=local_con,
                  index=False)

## 3.5 Updating the weather and flight data on local database

In [ ]:
update_weather_flights(local_con)

## 4.1 Transferring data from local to cloud

In [ ]:
# Importing the dataframes from local SQL
city_df, population_df, airports_df, cities_airports_df = fetch_static_data(local_con)
weather_df, flights_df = fetch_dynamic_data(local_con)

In [ ]:
# Exporting the dataframes into cloud SQL
submit_static_data(city_df, population_df, airports_df, cities_airports_df, cloud_con)
submit_weather_data(weather_df, cloud_con)
submit_flights_data(flights_df, cloud_con)